In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.3 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import json
import os
import re
from typing import Tuple, Dict, Any, List

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

from transformers import GPT2Model, AutoTokenizer, UniSpeechSatModel, AutoFeatureExtractor
from huggingface_hub import notebook_login
from datasets import load_dataset

In [3]:
notebook_login()

In [5]:
dataset = load_dataset('danielroncel/dstc2_audios')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25516 [00:00<?, ? examples/s]

In [6]:
feature_extractor = AutoFeatureExtractor.from_pretrained("microsoft/unispeech-sat-base")

preprocessor_config.json:   0%|          | 0.00/213 [00:00<?, ?B/s]

In [7]:
def repeat_audio(original_audio, target_duration):

    if len(original_audio) >= target_duration:
      return original_audio

    audio = original_audio.copy()
    while len(audio) < target_duration:
        audio = np.concatenate([audio, original_audio], axis=0)
    return audio

def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    paths = [x["path"] for x in examples["audio"]]

    audio_arrays = [repeat_audio(audio, feature_extractor.sampling_rate * 3.0) for audio in audio_arrays]

    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        padding='max_length',
        max_length=int(feature_extractor.sampling_rate * 3.0),
        truncation=True,
    )

    inputs = inputs['input_values']

    return {'path': paths, 'input_values': inputs}

In [8]:
encoded_dataset = dataset.map(preprocess_function, batched=True)
encoded_dataset

Map:   0%|          | 0/25516 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['audio', 'session_ids', 'path', 'input_values'],
        num_rows: 25516
    })
})

In [9]:
encoded_dataset = encoded_dataset.remove_columns(["audio"])
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['session_ids', 'path', 'input_values'],
        num_rows: 25516
    })
})

In [10]:
encoded_dataset.push_to_hub('danielroncel/dstc2_audios_input_uniSpeechSAT')

Uploading the dataset shards:   0%|          | 0/10 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/danielroncel/dstc2_audios_input_uniSpeechSAT/commit/471c00171a66d485a71e16ee0c4c25da9467605c', commit_message='Upload dataset', commit_description='', oid='471c00171a66d485a71e16ee0c4c25da9467605c', pr_url=None, pr_revision=None, pr_num=None)